In [5]:
from main import *
import time

experience = create_experience("main.py -name mytest -device cpu -num_epochs 50 -no_reg")

parameters = experience["parameters"]
experiment = experience["experiment"]
listener = experience["listener"]
log = experience["log"]
optimizer = experience["optimizer"]
losses = experience["losses"]
model = experience["model"] 
scheduler = experience["scheduler"]

parameters.disp()

run_experience(experiment, model, parameters, losses, optimizer, scheduler, listener, log)

print()

experiment.input_image.data.clamp_(0, 1)

log.info("Done style transfering over "+str(experiment.epoch)+" epochs!")

if parameters.save_model:
    save_all(experiment,model,parameters,listener)
if not(parameters.ghost):
    save_images(parameters.res_dir+"output.png",experiment.style_image,experiment.input_image,experiment.content_image)
else:
    plt_images(experiment.style_image,experiment.input_image,experiment.content_image)

if not(parameters.no_metrics):
    generate_plots(parameters, listener)

print("All done")

You have entered an experiment name that already exists even though you are not resuming that experiment, do you wish to continue (this will delete the folder: experiments/mytest/). [y/n] y
----- Experiments Parameters -----
name : mytest
quick : False
ghost : False
no_metrics : False
style_image : 1
content_image : 1
keep_params : False
base_model : vgg19
device : cpu
content_layers : ['conv4_2']
style_layers : ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1']
num_epochs : 50
style_weight : 1000000.0
content_weight : 10000.0
reg_weight : 0.0001
optimizer : lbfgs
lr : 1
momentum : 0
weight_decay : 0
scheduler : plateau
lr_step : 100
lr_decay : 0
seed : 42
no_log : False
reg : False
save_model : True
resume_model : False
load_name : 
imsize : (128, 128)
verbose : True
work_dir : 
save_name : mytest
res_dir : experiments/mytest/
load_path : experiments//save/
load_model_path : experiments//save/model.pt
load_parameters_path : experiments//save/parameters.json
load_experiment_path : experiment

AttributeError: 'int' object has no attribute 'item'